In [ ]:
import neat
from FieldNeat.Organism import Organism
from FieldNeat.EnvNeat import Environment

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "FieldNeat/config-neat")
p = neat.Population(config)
organisms = [Organism(1, 1, p.population[key], config) for key in p.population.keys()]
env = Environment(organisms, config)
#
# while True:
#     env.update()
#     env.render()

for i in range(5_000):
    if i % 5_000 == 0:
        print(i)

    env.update()
    if i > 4_000:
        env.render()


In [39]:
test = [1, 2, 3]

In [40]:
best = test[0]

In [41]:
test[0] = 100

In [42]:
best

1

In [4]:
from Field import GridWorld
import neat
import numpy as np

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "config-neat")
p = neat.Population(config)
population = list(p.population.values())
for genome in population:
    genome.fitness = 0
best_organism = population[0]
best_organism_idx = 0
env = GridWorld(width=10, height=10, nr_agents=len(p.population))

In [5]:
s = env.reset()
env.max_step = 10_000
brains = [neat.nn.FeedForwardNetwork.create(population[index], config) for index in range(len(population))]
max_age = 0
for n_epi in range(10_000):
    if n_epi % 10_000 == 0:
        print(n_epi)
    
    
    actions = []
    
    for index in range(len(population)):
        
        action = np.argmax(brains[index].activate(s[index]))
        actions.append(action)

    s_prime, r, dones, infos = env.step(actions)

    highest_age = max([agent.age for agent in env.agents])
    if highest_age > max_age:        
        max_age = highest_age
    
    
    for index in range(len(p.population)):
        if dones[index]:
            parent1 = best_organism
            parent2 = best_organism

            fitnesses = [agent.age for agent in env.agents]
            if env.agents[best_organism_idx].age < max(fitnesses):
                parent1 = population[np.argmax(fitnesses)]
                parent2 = population[np.argmax(fitnesses)]
                best_organism = population[np.argmax(fitnesses)]
                best_organism_idx = np.argmax(fitnesses)

            child = config.genome_type(1)
            child.configure_crossover(parent1, parent2, config.genome_config)
            child.mutate(config.genome_config)
            child.fitness = 0
            brains[index] = neat.nn.FeedForwardNetwork.create(population[index], config)
            population[index] = child
            
            env.agents[index].x, env.agents[index].y = env.agents[index].x, env.agents[index].y
            env.agents[index].health = 200
            env.agents[index].value = env.agents[index].value
            env.agents[index].coordinates = [env.agents[index].x, env.agents[index].y]
            env.agents[index].target_coordinates = [env.agents[index].x, env.agents[index].y]
            env.agents[index].dead = False
            env.agents[index].done = False
            env.agents[index].age = 0
            env.agents[index].x_target = None
            env.agents[index].y_target = None

0


In [8]:
s = env.reset()

In [24]:
env.grid.grid

array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [3., 0., 1., 0., 0., 0., 0., 0., 0., 2.],
       [0., 0., 0., 0., 0., 0., 0., 3., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 2., 2., 0., 3., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0., 2., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

In [25]:
actions = []
for index in range(len(population)):
    action = np.argmax(brains[index].activate(s[index]))
    actions.append(action)
actions

[3, 3, 3]

In [26]:
s, r, dones, infos = env.step(actions)
env.grid.grid

array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 2., 2., 3., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0., 2., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])